#### Ex 1:  Compress
Scrivere una funzione `compress` che, presa una lista, elimini gli elementi duplicati consecutivi

```
compress ["a"; "a"; "a"; "a"; "b"; "c"; "c"; "a"; "a"; "d"; "e"; "e"; "e"; "e"];;
- : string list = ["a"; "b"; "c"; "a"; "d"; "e"]
```

In [1]:
(*Dovendo scartare elementi duplicati consecutivi, 
  possiamo sfruttare un pattern che riconosca 
  i primi *due* elementi della lista*)
let rec compress list = 
    match list with
    |[] -> []
    |[x] -> [x]
    |x::y::t -> if x <> y 
                (*se sono distinti, conservo x
                  e mi richiamo ricorsivamente*)
                then x::(compress (y::t)) 
                (*se sono uguali, scarto x,
                  e mi richiamo ricorsivamente*)
                else (compress (y::t));;
                (*se effettuassi la chiamata ricorsiva su t, e non su y::t,
                  perderei l'informazione di y. Quindi il pattern matchin riiconosce
                  gli elementi x e y, ma ogni chiamata ricorsiva opera solo sull'
                  elemento x*)

compress ["a"; "a"; "a"; "a"; "b"; "c"; "c"; "a"; "a"; "d"; "e"; "e"; "e"; "e"];;

val compress : 'a list -> 'a list = <fun>


- : string list = ["a"; "b"; "c"; "a"; "d"; "e"]


#### Ex 2: Run Length Encoding

Scrivere una funzione `rle` che effettui il **Run Length Encoding** di una lista (https://www.geeksforgeeks.org/run-length-encoding/). 
```
rle ["a"; "a"; "a"; "a"; "b"; "c"; "c"; "a"; "a"; "d"; "e"; "e"; "e"; "e"];;
- : (int * string) list =
[("a", 4); ("b", 1); ("c", 2); ("a", 2); ("d", 1); ("e", 4)]
```

In [2]:
(* Come nella funzione precedente, possiamo usare un 
  pattern che riconosca i primi 2 elementi della lista,
  per poter controllare se sono uguali o distinti.
  Questa informazione però non è sufficiente, è necessario anche
  sapere quanti caratteri uguali si sono incontrati fin'ora,
  per questo va usato un parametro aggiuntivo count.
*)
let rle list = 
    let rec aux list count = 
        match list with
        |[] -> []
        |[x] -> [(x, count+1)] (*ho incontrato count occorrenze di x prima di ora*)
        (*grazie al costrutto as, definisco 4 variabili in una volta sola:
        x è il primo elemento, y il secondo, t è la *lista* (forse vuota)
        di elementi dopo y, e rest è definito come y::t, cioè la lista di elementi
        da y in poi*)
        |x::(y::t as rest) -> if x = y 
                            (*se sono uguali, non devo aggiungere x al risultato,
                              devo solo aumentare count*)    
                            then (aux rest (count+1))
                            (*se sono diversi, devo aggiungere x al risultato,
                              sotto forma di coppia (elemento * numero di occorrenze)
                              Dopo di che continuo a percorrere la lista resettando
                              count a 0*)
                            else (x, count+1)::(aux rest 0)
    in
    aux list 0;;

rle ["a"; "a"; "a"; "a"; "b"; "c"; "c"; "a"; "a"; "d"; "e"; "e"; "e"; "e"];;

val rle : 'a list -> ('a * int) list = <fun>


- : (string * int) list =
[("a", 4); ("b", 1); ("c", 2); ("a", 2); ("d", 1); ("e", 4)]


#### Ex 3: Ordinare le liste per frequenza

Data una lista di liste, come `let l = [[1;1;1;1]; [1; 2]; [1;2;3;4;5]; [1]; [3;3;3;3]; [6; 7]]`, a ogni elemento `e` di `l` è associata una lunghezza (`List.length e`). Diciamo che una lunghezza `x` è più **frequente** di una lunghezza `y` se ci sono più elementi di lunghezza `x` che elementi di lunghezza `y`. Ad esempio in `l`, ci sono 2 liste di lunghezza 4 e solo una lista di lunghezza 1, quindi 4 è più frequente di 1.

Scrivere una funzione `sort_by_frequency` che, presa una lista di liste come input, ordini quella lista per frequenza crescente delle lunghezze, ovvero mettendo prima le liste con lunghezze più rare (i.e. con una frequenza più bassa) e poi quelle con lunghezze più comuni.

```
sort_by_frequency [[1;1;1;1]; [1; 2]; [1;2;3;4;5]; [1]; [3;3;3;3]; [6; 7]];;
- : int list list =
[[1; 2; 3; 4; 5]; [1]; [1; 1; 1; 1]; [1; 2]; [3; 3; 3; 3]; [6; 7]]
```

Hint: Una volta ottenuta una lista di tutte le lunghezze delle liste in `l`, applicando la funzione di RLE dell'esercizio precedente si ottiene una lista di coppie, che associa a ogni lunghezza `x` la sua frequenza `f`. Consultando questa lista di coppie è possibile associare a ogni lista in `l` la frequenza della propria lunghezza. Le funzioni List.sort e List.assoc possono essere utili.

In [4]:
(*
  L'algoritmo che vogliamo implementare è composto da due parti
   1) Ottenere, per ogni lunghezza la frequenza sua frequenza
   2) Ottenere, per ogni lista, la frequenza della lunghezza della lista
   3) Ordinare le liste in base alla frequenza
  
  Per implementare il primo passo, dobbiamo sapere la frequenza di ogni lunghezza,
  ovvero sapere quante occorrenze ha una lunghezza. Possiamo costruire una lista "length"
  che contenga, anzichè altre liste, solo la lunghezza di queste liste. Per contare quindi
  quante occorrenze abbia ciascun elemento di "lengths", sono possibili 2 approcci:
   -Ordinare la lista, così da avere gli elementi duplicati affiancati,
    e poterli quindi contare in una seconda passata. (Complessità O(nlogn) in tempo)
   -Effettuare un'unica passata della lista, usando una hashmap per sapere quante volte
    è stato visitato ogni elemento. (Complessità O(n) in tempo, ma O(n) anche in spazio)
  In questo esercizio useremo il primo approccio, ordinano "lengths" e poi contando le 
  occorrenze di ogni elemento, Ottieniamo così una lista di coppie "frequencies", dove
  ogni coppia è costituita da (lunghezza, frequenza).
  
  Per implementare il secondo passo, data la lista "frequencies", dobbiamo sapere di una lista "l"
  quale è la frequenza della sua lunghezza, ovverro dobbiamo calcolare la lunghezza di l e
  cercare a quale frequenza è associata quella lunghezza (i.e. dobbiamo cercare una coppia
  (len, freq) all'interno di frequencies, tale che len sia proprio la lunghezza di l)

  Per implementare il terzo passo, basta utilizzare la funzione List.sort. Essa si aspetta in input
  una lista da ordinare e una funzione binaria f, tale che se a < b, f a b sia minore di zero,
  se a > b allora f a b sia maggiore di zero etc. Quindi, volendo ordinare le liste in base alla frequenza,
  dobbiamo scrivere una funzione f che, date due liste l1 e l2, restituisca un numero < 0 se la frequenza di l1
  è minore della frequenza di l2, e così via
*)

let sort_by_frequency lists = 
    (*-----PASSO 1--------*)
    (*Usiamo la funzione map per sostituire ogni lista in lists con la sua lunghezza*)
    let lengths = List.map (fun list -> List.length list) lists in
    (*Usiamo la funzione sort con il parametro compare, dove compare è una funzione tale che:
      se a < b, compare a b <0, se a > b allora compare a b > 0, se a = b,
      compare a b = 0
      Usare List.sort compare su una lista di interi vuol dire quindi ordinarli secondo
      la normale nozione di ordinamento "<"*)
    let sorted_lengths = List.sort compare lengths in
    (*Ora che le lunghezze uguali sono affiancate, basta eseguire la funzione rle definita prima
      per ottenere una associazione fra lunghezze e frequenze.
      Ovvero, si passa da 
          [1; 1; 1; 2; 3; 4; 4; 4;] a [(1, 3); (2, 1); (3, 1); (4, 3);]
      dove ogni lunghezza si trova in una coppia insieme alla sua frequenza 
      (cioè insieme al suo numero di occorrenze)*)
    let frequencies = rle sorted_lengths in
    
    (*-----PASSO 2--------*)
    let get_frequency list = 
      (*cerca la lunghezza length dentro la lista di coppie
        association, e restituisce la frequenza*)
      let rec search_frequency length association = 
       match association with
        |[] -> failwith "Non dovremmo mai raggiungere la fine di frequencies"
        |(len, freq)::t -> if len = length then freq
                           else search_frequency length t
      in
      
      (*calcoliamo la lunghezza di list e poi cerca a quale
        frequenza è associata quella lunghezza*)
      let length = List.length list in
      search_frequency length frequencies
    in
    
    (*-----PASSO 3--------*)
    (*Ordiniamo le liste all'interno di lists in base alla loro frequenza,
      ovvero ordinandole secondo compare sulla frequenza della lunghezza 
      di ogni lista*)
    List.sort (
      fun l1 l2 -> compare (get_frequency l1) (get_frequency l2)
    ) lists;;
    
sort_by_frequency [[1;1;1;1]; [1; 2]; [1;2;3;4;5]; [1]; [3;3;3;3]; [6; 7]];;
    

val sort_by_frequency : 'a list list -> 'a list list = <fun>


- : int list list =
[[1; 2; 3; 4; 5]; [1]; [1; 1; 1; 1]; [1; 2]; [3; 3; 3; 3]; [6; 7]]
